Notebook Settings
=================

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run ../../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'
```

Imports
=======

``` ipython
import sys
sys.path.insert(0, '../../')

import torch
import pandas as pd
from time import perf_counter

from src.network import Network
from src.lif_network import LIFNetwork
from src.plot_utils import plot_con
from src.decode import decode_bump
```

Helpers
=======

Connectivity
------------

``` ipython
def plot_eigen(W):
    # Compute eigenvalues
    eigenvalues = torch.linalg.eigvals(W).cpu().numpy()

    # Extract real and imaginary parts
    real_parts = eigenvalues.real
    imag_parts = eigenvalues.imag

    # Plotting
    plt.scatter(real_parts, imag_parts)
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.axvline(x=0, color='k', linestyle='--')

    # plt.grid(True, which='both')
    plt.show()
```

Random
------

``` ipython
def convert_seconds(seconds):
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return h, m, s
```

``` ipython
def get_theta(a, b, GM=0, IF_NORM=0):

    if GM:
        b = b - np.dot(b, a) / np.dot(a, a) * a

    if IF_NORM:
        u = a / np.linalg.norm(a)
        v = b / np.linalg.norm(b)
    else:
        u=a
        v=b

    return np.arctan2(v, u)
```

``` ipython
def normalize(v):
    return v / np.linalg.norm(v)

def project(x, u):
    return x * u
# return np.dot(x, u) * u

def sort_by_angle(x, u, v):
    u_hat = normalize(u)
    v_hat = normalize(v)

    x_proj_u = project(x, u_hat)
    x_proj_v = project(x, v_hat)
    # x_proj = x_proj_u + x_proj_v
    theta = np.arctan2(x_proj_v, x_proj_u) + np.pi

    # cos_theta = np.dot(x_proj, u_hat) / np.linalg.norm(x_proj) * u_hat
    # sin_theta = np.dot(x_proj, v_hat) / np.linalg.norm(x_proj) * v_hat
    # theta = np.arctan2(sin_theta, cos_theta)

    # Pair up each element of x with the corresponding angle
    # x_angle_pairs = list(zip(x, theta))

    # Sort based on the angle
    # x_angle_pairs.sort(key=lambda pair: pair[1])

    # Extract the sorted elements
    # sorted_x = [pair[0] for pair in x_angle_pairs]

    return theta
```

``` ipython
def get_idx(model):
    ksi = model.PHI0.cpu().detach().numpy()
    print(ksi.shape)

    idx = np.arange(0, len(ksi[0]))
    theta = get_theta(ksi[0], ksi[2], GM=0, IF_NORM=0)

    return theta.argsort()
```

``` ipython
def get_overlap(model, rates):
    ksi = model.PHI0.cpu().detach().numpy()
    return rates @ ksi.T / rates.shape[-1]

```

Connectivity
============

Imports
-------

``` ipython
from src.connectivity import Connectivity
```

Sparse Matrix
-------------

### Sparse Matrix

``` ipython
Wij = Connectivity(1000, 1000, 100)('sparse', 'None', kappa=1.0, sigma=0, phase=0)
```

``` ipython
Cij = Wij.cpu().detach().numpy()
print(Cij.shape)
```

``` ipython
plt.figure(figsize=(12, 5))  # Set the figure size (width, height) in inches

ax1 = plt.subplot2grid((2, 3), (0, 0), rowspan=2)
im = ax1.imshow(Cij, cmap='jet', aspect=1)
ax1.set_xlabel("Presynaptic")
ax1.set_ylabel("Postsynaptic")

# Second column, first row
ax2 = plt.subplot2grid((2, 3), (0, 1))
Kj = np.sum(Cij, axis=0)  # sum over pres
ax2.plot(Kj)
# ax2.set_xticklabels([])
ax2.set_ylabel("$K_j$")

# # Second column, second row
ax3 = plt.subplot2grid((2, 3), (1, 1))
Ki = np.sum(Cij, axis=1)  # sum over pres
ax3.plot(Kj)
ax3.set_ylabel("$K_i$")

ax4 = plt.subplot2grid((2, 3), (0, 2), rowspan=2)
diags = []
for i in range(int(Cij.shape[0] / 2)):
    diags.append(np.trace(Cij, offset=i) / Cij.shape[0])
diags = np.array(diags)
ax4.plot(diags)
ax4.set_xlabel("Neuron #")
ax4.set_ylabel("$P_{ij}$")

plt.tight_layout()
plt.show()

```

``` ipython
plot_eigen(1.0 * Wij)
```

### Sparse Matrix with Cosine Structure

``` ipython
Wij = Connectivity(1000, 1000, 100)('sparse', 'cosine', kappa=1.0, sigma=0, phase=0)
```

``` ipython
Cij = Wij.cpu().detach().numpy()
print(Cij.shape)
```

``` ipython
plt.figure(figsize=(12, 5))  # Set the figure size (width, height) in inches

ax1 = plt.subplot2grid((2, 3), (0, 0), rowspan=2)
im = ax1.imshow(Cij, cmap='jet', aspect=1)
ax1.set_xlabel("Presynaptic")
ax1.set_ylabel("Postsynaptic")

# Second column, first row
ax2 = plt.subplot2grid((2, 3), (0, 1))
Kj = np.sum(Cij, axis=0)  # sum over pres
ax2.plot(Kj)
# ax2.set_xticklabels([])
ax2.set_ylabel("$K_j$")

# # Second column, second row
ax3 = plt.subplot2grid((2, 3), (1, 1))
Ki = np.sum(Cij, axis=1)  # sum over pres
ax3.plot(Kj)
ax3.set_ylabel("$K_i$")

ax4 = plt.subplot2grid((2, 3), (0, 2), rowspan=2)
diags = []
for i in range(int(Cij.shape[0] / 2)):
    diags.append(np.trace(Cij, offset=i) / Cij.shape[0])
diags = np.array(diags)
ax4.plot(diags)
ax4.set_xlabel("Neuron #")
ax4.set_ylabel("$P_{ij}$")

plt.tight_layout()
plt.show()

```

``` ipython
plot_eigen(1.0 * Wij)
```

``` ipython
```

### Sparse Matrix with Low Rank Structure

``` ipython
Con = Connectivity(1000, 1000, 100, verbose=1)
Wij = Con('sparse', 'lr', kappa=5, lr_mean=[0, 0], lr_cov=[[1, 0], [0, 1]])
```

``` ipython
Cij = Wij.cpu().detach().numpy()
print(Cij.shape)
```

``` ipython
plt.figure(figsize=(12, 5))  # Set the figure size (width, height) in inches

ax1 = plt.subplot2grid((2, 3), (0, 0), rowspan=2)
im = ax1.imshow(Cij, cmap='jet', aspect=1)
ax1.set_xlabel("Presynaptic")
ax1.set_ylabel("Postsynaptic")

# Second column, first row
ax2 = plt.subplot2grid((2, 3), (0, 1))
Kj = np.sum(Cij, axis=0)  # sum over pres
ax2.plot(Kj)
# ax2.set_xticklabels([])
ax2.set_ylabel("$K_j$")

# # Second column, second row
ax3 = plt.subplot2grid((2, 3), (1, 1))
Ki = np.sum(Cij, axis=1)  # sum over pres
ax3.plot(Kj)
ax3.set_ylabel("$K_i$")

ax4 = plt.subplot2grid((2, 3), (0, 2), rowspan=2)
diags = []
for i in range(int(Cij.shape[0] / 2)):
    diags.append(np.trace(Cij, offset=i) / Cij.shape[0])
diags = np.array(diags)
ax4.plot(diags)
ax4.set_xlabel("Neuron #")
ax4.set_ylabel("$P_{ij}$")

plt.tight_layout()
plt.show()

```

``` ipython
plot_eigen(1.0 * Wij)
```

``` ipython
ksi = Con.ksi.cpu().detach().numpy()
print('ksi', ksi.shape)
idx = np.arange(0, len(ksi[0]))
# print(theta.shape)
theta = get_theta(ksi[0], ksi[1], GM=0, IF_NORM=1)
theta = np.arctan2(ksi[1], ksi[0])
index_order = theta.argsort()
# print(index_order)
Cij_ordered = Cij[:][index_order]
print(Cij_ordered.shape)
```

``` ipython
plt.figure(figsize=(12, 5))  # Set the figure size (width, height) in inches

ax1 = plt.subplot2grid((2, 3), (0, 0), rowspan=2)
im = ax1.imshow(Cij_ordered, cmap='jet', aspect=1)
ax1.set_xlabel("Presynaptic")
ax1.set_ylabel("Postsynaptic")

# Second column, first row
ax2 = plt.subplot2grid((2, 3), (0, 1))
Kj = np.sum(Cij_ordered, axis=0)  # sum over pres
ax2.plot(Kj)
# ax2.set_xticklabels([])
ax2.set_ylabel("$K_j$")

# # Second column, second row
ax3 = plt.subplot2grid((2, 3), (1, 1))
Ki = np.sum(Cij_ordered, axis=1)  # sum over pres
ax3.plot(Kj)
ax3.set_ylabel("$K_i$")

ax4 = plt.subplot2grid((2, 3), (0, 2), rowspan=2)
diags = []
for i in range(int(Cij_ordered.shape[0] / 2)):
    diags.append(np.trace(Cij_ordered, offset=i) / Cij_ordered.shape[0])
diags = np.array(diags)
ax4.plot(diags)
ax4.set_xlabel("Neuron #")
ax4.set_ylabel("$P_{ij}$")

plt.tight_layout()
plt.show()

```

``` ipython
```

### Using torch.sparse

``` ipython
REPO_ROOT = "/home/leon/models/NeuroFlame"
model = Network('config_EI.yml', 'test', REPO_ROOT, VERBOSE=1, DEVICE='cuda', TASK='odr',
                PROBA_TYPE=['cosine', '', '',''], KAPPA=[1, 0, 0, 0])
```

``` ipython
import torch

# Simulate a large dense matrix
# Example is small for demonstration purposes, adjust sizes accordingly
# dense_matrix = torch.tensor([[0, 0, 3], [4, 0, 0], [0, 0, 5]], dtype=torch.float32)

N = 10000
K = 1000

# dense_matrix = 1.0 * (torch.rand(N, N, device='cuda') <= (K / float(N)))
dense_matrix = model.Wab_T.T

# Define variables to store indices and values of non-zero elements
nnz_indices = []
nnz_values = []

# Define chunk size (adjust based on your memory constraints)
chunk_size = 4  # Here, a chunk consists of 1 row for simplicity

# Loop through chunks of the matrix
for i in range(0, dense_matrix.size(0), chunk_size):
    # Get the current chunk
    chunk = dense_matrix[i:i+chunk_size, :]

    # Find non-zero elements in the chunk
    chunk_nnz_indices = torch.nonzero(chunk, as_tuple=False).t()  # Transpose to match COO format
    chunk_nnz_values = chunk[chunk_nnz_indices[0], chunk_nnz_indices[1]]

    # Adjust chunk indices to global indices
    chunk_nnz_indices[0] += i  # Adjust row indices for chunks beyond the first

    # Append current chunk's non-zero elements to the lists
    nnz_indices.append(chunk_nnz_indices)
    nnz_values.append(chunk_nnz_values)

# Concatenate all non-zero indices and values
nnz_indices = torch.cat(nnz_indices, dim=1)  # Concatenate along columns
nnz_values = torch.cat(nnz_values)

# Create sparse tensor
sparse_matrix = torch.sparse_coo_tensor(nnz_indices, nnz_values, dense_matrix.size())

print(sparse_matrix)
```

``` ipython
plot_con(sparse_matrix.to_dense().cpu().detach().numpy().T)
```

Full Matrix
-----------

### All to All with Von Mises Shape

``` ipython
Cij = Connectivity(1000, 1000, 1.0)('all2all', 'von_mises', kappa=1.0, sigma=0, phase=0)
```

``` ipython
Cij = Cij.cpu().detach().numpy()
print(Cij.shape)
```

``` ipython
plt.figure(figsize=(12, 5))  # Set the figure size (width, height) in inches

ax1 = plt.subplot2grid((2, 3), (0, 0), rowspan=2)
im = ax1.imshow(Cij, cmap='jet', aspect=1)
ax1.set_xlabel("Presynaptic")
ax1.set_ylabel("Postsynaptic")

# Second column, first row
ax2 = plt.subplot2grid((2, 3), (0, 1))
Kj = np.sum(Cij, axis=0)  # sum over pres
ax2.plot(Kj)
# ax2.set_xticklabels([])
ax2.set_ylabel("$K_j$")

# # Second column, second row
ax3 = plt.subplot2grid((2, 3), (1, 1))
Ki = np.sum(Cij, axis=1)  # sum over pres
ax3.plot(Kj)
ax3.set_ylabel("$K_i$")

ax4 = plt.subplot2grid((2, 3), (0, 2), rowspan=2)
diags = []
for i in range(int(Cij.shape[0] / 2)):
    diags.append(np.trace(Cij, offset=i) / Cij.shape[0])
diags = np.array(diags)
ax4.plot(diags)
ax4.set_xlabel("Neuron #")
ax4.set_ylabel("$P_{ij}$")

plt.tight_layout()
plt.show()

```